In [1982]:
import re
import numpy
from matplotlib import pyplot

file = open('data/22.txt', 'r')
data = file.read()

file.close()

In [1983]:
lines, path = [], []

for line in data.splitlines():
    if not line:
        continue

    if parsed := re.findall(r'(\d+)([RL]?)', line):
        path = parsed
        continue

    lines.append(line)

In [1984]:
convert = lambda c: ord(c) - ord(' ')
wall = convert('#')

xmax, ymax = max(map(len, lines)), len(lines)
field = numpy.zeros((ymax, xmax), numpy.int16)
boundaries = numpy.zeros((2, max(ymax, xmax), 2), numpy.int32)

rotations = {
    '' : [[ 1,  0], [ 0,  1]], # 0
    'R': [[ 0, -1], [ 1,  0]], # 90
    'L': [[ 0,  1], [-1,  0]], # 270
    'T': [[-1,  0], [ 0, -1]], # 180
}

In [1985]:
# Field
for y, line in enumerate(lines):
    field[y, :len(line)] = list(map(convert, line))

# ymin, ymax boundaries for each x
for x in range(xmax):
    filled = numpy.flatnonzero(field[:, x])
    boundaries[0, x] = [min(filled), max(filled)]

# xmin, xmax boundaries for each y
for y in range(ymax):
    filled = numpy.flatnonzero(field[y, :])
    boundaries[1, y] = [min(filled), max(filled)]

In [1986]:
def rollaxis(x, bmin, bmax):
    """Respawn on the other end if out of bounds"""
    if x > bmax: return bmin
    if x < bmin: return bmax
    return x

    #return (x - bmin) % (bmax - bmin + 1) + bmin

def wrap(y, x, direction):
    """Move in the direction and check OOB"""
    y, x = [y, x] + direction
    y = rollaxis(y, *boundaries[0, x]) if direction[0] else y
    x = rollaxis(x, *boundaries[1, y]) if direction[1] else x

    return y, x

def facing(y, x):
    """Get `facing` number of the direction"""
    return abs(y * 2 + x - 1)

def password(y, x, direction):
    """Get `password` for given state"""
    print (1000 * (y + 1) + 4 * (x + 1) + facing(*direction))

def walk(position, direction, path):
    """Go through the path from starting position and direction to the end"""
    for steps, rotate in path:
        for _ in range(int(steps)):
            y, x = wrap(*position, direction)

            if field[y, x] == wall:
                break

            position = [y, x]

        if rotate:
            direction = numpy.matmul(direction, rotations[rotate])

    return position, direction

In [1987]:
position = [0, boundaries[1, 0, 0]]
direction = numpy.array([0, 1], numpy.int8)

position, direction = walk(position, direction, path)
password(*position, direction)

95358


In [1988]:
size = 0,
planes = []
rotations3d = []

In [1989]:
def to_local(y, x, plane):
    [ymin, _], [xmin, _] = planes[plane]
    return y - ymin, x - xmin

def from_local(y, x, plane):
    [ymin, _], [xmin, _] = planes[plane]
    return y % size + ymin, x % size + xmin

def get_plane(y, x):
    for i in range(1, len(planes)):
        if (planes[i][0][0] <= y <= planes[i][0][1] and 
            planes[i][1][0] <= x <= planes[i][1][1]):
            return i

    return None

def rotcoord(y, x, rot):
    rotvec = numpy.zeros((size, size), numpy.int8)
    rotvec[y, x] = 1

    if rot == 'L':
        rotvec = numpy.rot90(rotvec, axes=(0, 1))
    elif rot == 'T':
        rotvec = numpy.rot90(rotvec, 2)
    elif rot == 'R':
        rotvec = numpy.rot90(rotvec, axes=(1, 0))

    return numpy.argwhere(rotvec)[0]

def rolllocal(x):
    return rollaxis(x, 0, size - 1)

def wrap3d(y, x, direction):
    plane = get_plane(y, x)
    local = to_local(y, x, plane)
    new = local + direction

    for axis in range(2):
        if direction[axis]:
            wrapped = rolllocal(new[axis])

            if new[axis] != wrapped:
                new[axis] = wrapped
                plane, rot = rotations3d[axis][plane][0 if direction[axis] > 0 else 1]
                direction = numpy.matmul(direction, rotations[rot])
                new = rotcoord(*new, rot)

    y, x = from_local(*new, plane)
    return y, x, direction

def walk3d(position, direction, path):
    """Go through the path from starting position and direction to the end"""
    for steps, rotate in path:
        for _ in range(int(steps)):
            y, x, _direction = wrap3d(*position, direction)

            if field[y, x] == wall:
                break

            direction = _direction
            position = [y, x]

        if rotate:
            direction = numpy.matmul(direction, rotations[rotate])

    return position, direction

In [1990]:
size = 50
position = [0, 50]
direction = numpy.array([0, 1], numpy.int8)

planes = [
    [],
    # ymin, ymax, xmin, xmax
    [[0,        size - 1    ], [size,     size * 2 - 1]],
    [[size,     size * 2 - 1], [size,     size * 2 - 1]],
    [[size * 2, size * 3 - 1], [size,     size * 2 - 1]],
    [[size * 2, size * 3 - 1], [0,        size - 1    ]],
    [[size * 3, size * 4 - 1], [0,        size - 1    ]],
    [[0,        size - 1    ], [size * 2, size * 3 - 1]]
]

rotations3d = [
        # Up, down
    [
        [],
        [[2, '' ], [5, 'R']], # 1
        [[3, '' ], [1, '' ]], # 2
        [[5, 'R'], [2, '' ]], # 3
        [[5, '' ], [2, 'R']], # 4
        [[6, '' ], [4, '' ]], # 5
        [[2, 'R'], [5, '' ]]  # 6
    ],
        # Right, left
    [
        [],
        [[6, '' ], [4, 'T']], # 1
        [[6, 'L'], [4, 'L']], # 2
        [[6, 'T'], [4, '' ]], # 3
        [[3, '' ], [1, 'T']], # 4
        [[3, 'L'], [1, 'L']], # 5
        [[3, 'T'], [1, '' ]]  # 6
    ],
]

In [1991]:
position, direction = walk3d(position, direction, path)
password(*position, direction)

144361
